In [2]:
import json
import datetime
import logging
from confluent_kafka import Consumer, Producer

In [3]:
def validate_transaction(event):
    errors = []
    try:
        total_expected = event['amount'] + event['vat_amount'] + event['commission_amount']
        if event['total_amount'] != total_expected:
            errors.append('ERR_AMOUNT')
    except:
        errors.append('ERR_AMOUNT')

    try:
        now = datetime.datetime.utcnow()
        ts = datetime.datetime.fromisoformat(event['timestamp'].replace("Z", ""))
        day_per_sec = 86400
        if ts > now or (now - ts).total_seconds() > day_per_sec:
            errors.append('ERR_TIME')
    except:
        errors.append('ERR_TIME')

    try:
        if event['payment_method'] == 'mobile':
            os = event.get('device_info', {}).get('os', None)
            if os not in ['Android', 'iOS']:
                errors.append('ERR_DEVICE')
    except:
        errors.append('ERR_DEVICE')

    return errors

In [4]:
logging.basicConfig(level=logging.INFO)

consumer_conf = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'darooghe-consumer',
    'auto.offset.reset': 'earliest',
}
consumer = Consumer(consumer_conf)
consumer.subscribe(['darooghe.transactions'])

producer = Producer({'bootstrap.servers': 'localhost:9092'})

try:
    while True:
        msg = consumer.poll(1.0)
        if msg is None:
            continue
        if msg.error():
            logging.error(f"Consumer error: {msg.error()}")
            continue

        try:
            event = json.loads(msg.value().decode('utf-8'))
            # raw_event = json.loads(msg.value().decode('utf-8'))
            # event = parse_transaction(raw_event)
            transaction_id = event.get('transaction_id', 'unknown')

            errors = validate_transaction(event)

            if errors:
                error_msg = {
                    'transaction_id': transaction_id,
                    'errors': errors,
                    'original_data': event
                }
                producer.produce(
                    'darooghe.error_logs',
                    key=transaction_id,
                    value=json.dumps(error_msg, default=str)
                )
                logging.warning(f"Invalid transaction: {transaction_id}, errors: {errors}")
            else:
                logging.info(f"Valid transaction: {transaction_id}")

        except Exception as e:
            logging.error(f"Parse error: {e}")

        producer.poll(0)

except KeyboardInterrupt:
    logging.info("Shutting down...")

finally:
    consumer.close()
    producer.flush()

INFO:root:Valid transaction: a9a13eb1-35e5-41c1-b4b5-7c0d99e05d38
INFO:root:Valid transaction: 45ce56e7-7cc0-4466-85df-b7c1263781ad
INFO:root:Valid transaction: 8bd0fe05-4b8d-4074-a999-136ec2552702
INFO:root:Valid transaction: b4a116a7-5f18-4607-8fd7-1c70cdda94c6
INFO:root:Valid transaction: 4983457e-4196-45c3-9c32-71658a45c657
INFO:root:Valid transaction: fcc2d01f-8d9d-40f7-9e6f-fe8edbadcfe7
INFO:root:Valid transaction: 506671f0-c786-43c7-a918-5ae70a2cf3ec
INFO:root:Valid transaction: ee6de02e-871e-4daf-9e2c-dd5f3594de8d
INFO:root:Valid transaction: 74d40406-1c86-46d6-95c1-dfc0f9092be1
INFO:root:Valid transaction: 2044ae20-8d20-4331-8480-86037e71e4cb
INFO:root:Valid transaction: 2805588b-fdff-4a8c-81b3-5f2742e14d5b
INFO:root:Valid transaction: 81d862a0-ec57-42fd-a3c5-910cafc7da80
INFO:root:Valid transaction: 02196b37-eaf1-4ad8-bb3d-f8500c75e38a
INFO:root:Valid transaction: 15d3349c-4e78-4989-b160-a29ffb3eff50
INFO:root:Valid transaction: e94ea621-b217-4c29-b1bf-44f2d860fdfb
INFO:root:

In [9]:

consumer_conf = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'darooghe-consumer',
    'auto.offset.reset': 'earliest',
}
consumer = Consumer(consumer_conf)
consumer.subscribe(['darooghe.transactions'])

try:
    msg = consumer.poll(timeout=10.0)
    if msg is None:
        print("No message received.")
    elif msg.error():
        print(f"Error: {msg.error()}")
    else:
        # event = json.loads(msg.value().decode('utf-8'))
        raw_event = json.loads(msg.value().decode('utf-8'))
        event = parse_transaction(raw_event)
        print("Received message:")
        print(json.dumps(event, indent=2))

        errors = validate_transaction(event)
        if errors:
            print("Validation Errors:", errors)
        else:
            print("Message is VALID!")

finally:
    consumer.close()


Received message:
{
  "transaction_id": "0648a201-4d30-4e8d-914a-f6fa61b1e986",
  "timestamp": "2025-04-14T13:15:15.912374",
  "customer_id": "cust_478",
  "merchant_id": "merch_40",
  "merchant_category": "transportation",
  "payment_method": "mobile",
  "amount": 614135,
  "location": {
    "lat": 35.688137581485556,
    "lng": 51.35528624655932
  },
  "device_info": {
    "os": "iOS",
    "app_version": "3.1.0",
    "device_model": "iPhone 15"
  },
  "status": "approved",
  "commission_type": "flat",
  "commission_amount": 12282,
  "vat_amount": 55272,
  "total_amount": 681689,
  "customer_type": "individual",
  "risk_level": 2,
  "failure_reason": null
}
Message is VALID!


In [8]:
def parse_transaction(raw_event):
    try:
        event = {}
        event['transaction_id'] = str(raw_event['transaction_id'])
        event['timestamp'] = datetime.datetime.fromisoformat(
            raw_event['timestamp'].replace('Z', '')
        ).isoformat()
        event['customer_id'] = str(raw_event['customer_id'])
        event['merchant_id'] = str(raw_event['merchant_id'])
        event['merchant_category'] = str(raw_event['merchant_category'])
        event['payment_method'] = str(raw_event['payment_method'])
        event['amount'] = int(raw_event['amount'])
        event['location'] = {
            'lat': float(raw_event['location']['lat']),
            'lng': float(raw_event['location']['lng']),
        }
        event['device_info'] = raw_event.get('device_info', {})
        event['status'] = str(raw_event['status'])
        event['commission_type'] = str(raw_event['commission_type'])
        event['commission_amount'] = int(raw_event['commission_amount'])
        event['vat_amount'] = int(raw_event['vat_amount'])
        event['total_amount'] = int(raw_event['total_amount'])
        event['customer_type'] = str(raw_event['customer_type'])
        event['risk_level'] = int(raw_event['risk_level'])
        event['failure_reason'] = raw_event.get('failure_reason', None)
        return event
    except (KeyError, ValueError, TypeError) as e:
        print(f"Schema error: {e}")
        return None


# 2

### Collect data from Kafka (once) and save to a local file (e.g., JSON)